# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

In [1]:
from operator import itemgetter
import requests
import re
import bs4
import pandas as pd

In [2]:
def get_soup(url):
    resp = requests.get(url)
    return bs4.BeautifulSoup(resp.content, 'lxml')

In [3]:
soup = get_soup('http://books.toscrape.com/')

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [4]:
def retrieve_titles(soup):
    titles = pd.Series(soup.find_all('a', href=True, title=True))
    titles.name = 'titles'
    return titles.map(itemgetter('title'))

In [5]:
retrieve_titles(soup)

0                                  A Light in the Attic
1                                    Tipping the Velvet
2                                            Soumission
3                                         Sharp Objects
4                 Sapiens: A Brief History of Humankind
5                                       The Requiem Red
6     The Dirty Little Secrets of Getting Your Dream...
7     The Coming Woman: A Novel Based on the Life of...
8     The Boys in the Boat: Nine Americans and Their...
9                                       The Black Maria
10       Starving Hearts (Triangular Trade Trilogy, #1)
11                                Shakespeare's Sonnets
12                                          Set Me Free
13    Scott Pilgrim's Precious Little Life (Scott Pi...
14                            Rip it Up and Start Again
15    Our Band Could Be Your Life: Scenes from the A...
16                                                 Olio
17    Mesaerion: The Best Science Fiction Storie

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [6]:
from types import MappingProxyType
DIGIT_NAMES = ('zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine')
DIGIT_NAMES = MappingProxyType({x: DIGIT_NAMES.index(x) for x in DIGIT_NAMES})
DIGIT_NAMES

mappingproxy({'zero': 0,
              'one': 1,
              'two': 2,
              'three': 3,
              'four': 4,
              'five': 5,
              'six': 6,
              'seven': 7,
              'eight': 8,
              'nine': 9})

In [7]:
def retrieve_ratings(soup):
    ratings = pd.Series(soup.find_all('p', class_='star-rating'))
    ratings = (ratings.map(itemgetter('class'))
                .map(itemgetter(1))
                .str.lower()
                .map(DIGIT_NAMES))
    ratings.name = 'ratings'
    return ratings

In [8]:
retrieve_ratings(soup)

0     3
1     1
2     1
3     4
4     5
5     1
6     4
7     3
8     4
9     1
10    2
11    4
12    5
13    5
14    5
15    3
16    1
17    1
18    2
19    2
Name: ratings, dtype: int64

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [9]:
def retrieve_prices(soup):
    prices = pd.Series(soup.find_all('p', class_='price_color'))
    prices = (prices.map(lambda x: x.get_text())
                .str.slice(1)
                .astype(float))
    prices.name = 'prices'
    return prices

In [10]:
retrieve_prices(soup)

0     51.77
1     53.74
2     50.10
3     47.82
4     54.23
5     22.65
6     33.34
7     17.93
8     22.60
9     52.15
10    13.99
11    20.66
12    17.46
13    52.29
14    35.02
15    57.25
16    23.88
17    37.59
18    51.33
19    45.17
Name: prices, dtype: float64

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [11]:
def retrieve_availabilities(soup):
    avail = pd.Series(soup.find_all('p', class_='instock availability'))
    avail = avail.map(lambda x: x.get_text()).str.strip().str.lower()
    avail = avail == 'in stock'
    avail.name = 'available'
    return avail

In [12]:
retrieve_availabilities(soup)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: available, dtype: bool

In [13]:
len(soup.find_all('a'))

94

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [14]:
#Your code here
url_schema = 'http://books.toscrape.com/catalogue/category/books_1/page-{0}.html'
urls = [url_schema.format(x) for x in range(1, 51)]
urls[:5]

['http://books.toscrape.com/catalogue/category/books_1/page-1.html',
 'http://books.toscrape.com/catalogue/category/books_1/page-2.html',
 'http://books.toscrape.com/catalogue/category/books_1/page-3.html',
 'http://books.toscrape.com/catalogue/category/books_1/page-4.html',
 'http://books.toscrape.com/catalogue/category/books_1/page-5.html']

In [15]:
def extract_all_data(soup):
    funcs = (retrieve_titles,
                retrieve_ratings,
                retrieve_prices,
                retrieve_availabilities)
    return pd.concat([f(soup).to_frame() for f in funcs], axis=1)

In [16]:
df = pd.DataFrame()
for i, url in enumerate(urls):
    soup = get_soup(url)
    page_df = extract_all_data(soup)
    page_df.index = pd.MultiIndex.from_product(([i], page_df.index))
    page_df.index.names = ('page', 'pos')
    df = df.append(page_df)

In [17]:
df.head()

titles  ratings  prices  available
page pos                                                                   
0    0                     A Light in the Attic        3   51.77       True
     1                       Tipping the Velvet        1   53.74       True
     2                               Soumission        1   50.10       True
     3                            Sharp Objects        4   47.82       True
     4    Sapiens: A Brief History of Humankind        5   54.23       True

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [73]:
#Your code here
BASE = 'http://books.toscrape.com/catalogue/'

In [74]:
from urllib.parse import urljoin
def retrieve_next_page_url(soup):
    tag = soup.find('a', href=re.compile(r'page'), text='next')
    return urljoin(BASE, tag['href']) if tag else None

In [78]:
current_url = 'http://books.toscrape.com/catalogue/page-1.html'
soup = get_soup(current_url)
df = pd.DataFrame()
seen = set()
while current_url not in seen:
    df = df.append(extract_all_data(soup))
    seen.add(current_url)
    if retrieve_next_page_url(soup):
        current_url = retrieve_next_page_url(soup)
        soup = get_soup(current_url)

In [80]:
df.head()

,titles,ratings,prices,available
0,A Light in the Attic,3,51.77,True
1,Tipping the Velvet,1,53.74,True
2,Soumission,1,50.10,True
3,Sharp Objects,4,47.82,True
4,Sapiens: A Brief History of Humankind,5,54.23,True


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!